In [1]:
from __future__ import print_function, division, absolute_import

import argparse
import time

import numpy as np
seed = 7
np.random.seed(seed)

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
os.chdir('../CoMET')

import evolutron.networks as nets
from evolutron.motifs import motif_extraction
from evolutron.tools import load_dataset, none2str, Handle
from evolutron.engine import DeepTrainer

from keras.utils.np_utils import to_categorical

Using Theano backend.


In [2]:
# Data and model
data_id = 'dnabind'
padded = True
mode = 'family'

# Architecture
filters = 100
filter_length = 30
conv = 3
fc = 1

# Training
num_epochs = 1
batch_size = 50
optimizer = 'nadam'
rate = .01
validate = .2

In [3]:
dataset = load_dataset(data_id, padded=padded, codes=True)
x_data, y_data = dataset
if type(x_data) == np.ndarray:
    input_shape = x_data[0].shape
elif type(x_data) == list:
    input_shape = (None, x_data[0].shape[1])
else:
    raise TypeError('Something went wrong with the dataset type')

y_data = to_categorical(y_data)

print(x_data.shape)
print(y_data.shape)
output_dim = y_data.shape[1]

Dataset size: 11036
(11036, 1439, 20)
(11036, 399)


In [4]:
net_arch = nets.DeepCoFAM.from_options(input_shape,
                                               output_dim,
                                               n_conv_layers=conv,
                                               n_fc_layers=fc,
                                               n_filters=filters,
                                               filter_length=filter_length)
conv_net = DeepTrainer(net_arch)
conv_net.compile(optimizer=optimizer, lr=rate)

In [5]:
conv_net.display_network_info()

Neural Network has 710699 trainable parameters
  #  Name            Shape       Parameters
---  --------------  --------  ------------
  0  aa_seq          1439x20              0
  1  masking_1       1439x20              0
  2  Conv1           1439x100         60100
  3  Conv2           1439x100        300100
  4  Conv3           1439x100        300100
  5  maxpooling1d_1  1x100                0
  6  flatten_1       100                  0
  7  FCEnc1          100              10100
  8  Classifier      399              40299


In [6]:
conv_net.fit(x_data, y_data,
             nb_epoch=1,
             batch_size=batch_size,
             validate=validate)

Train on 8828 samples, validate on 2208 samples
Epoch 1/1


ValueError: Input dimension mis-match. (input[0].shape[1] = 50, input[1].shape[1] = 1439)
Apply node that caused the error: Elemwise{mul,no_inplace}(InplaceDimShuffle{x,0}.0, Elemwise{Cast{float32}}.0)
Toposort index: 195
Inputs types: [TensorType(float32, row), TensorType(float32, matrix)]
Inputs shapes: [(1, 50), (50, 1439)]
Inputs strides: [(200, 4), (5756, 4)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Sum{axis=[1], acc_dtype=float64}(Elemwise{mul,no_inplace}.0)]]

Debugprint of the apply node: 
Elemwise{mul,no_inplace} [id A] <TensorType(float32, matrix)> ''   
 |InplaceDimShuffle{x,0} [id B] <TensorType(float32, row)> ''   
 | |Sum{axis=[1], acc_dtype=float64} [id C] <TensorType(float32, vector)> ''   
 |   |Elemwise{mul,no_inplace} [id D] <TensorType(float32, matrix)> ''   
 |     |SoftmaxWithBias [id E] <TensorType(float32, matrix)> ''   
 |     | |Dot22 [id F] <TensorType(float32, matrix)> ''   
 |     | | |Elemwise{Composite{scalar_sigmoid((i0 + i1))}}[(0, 0

In [ ]:
import keras.backend as K

In [ ]:
pred = conv_net.predict(x_data[:5])

In [ ]:
pred.shape

In [ ]:
sc = conv_net.score(x_data[:5], y_data[:5])